<a id="toc"></a>
# Deimos source modelling
***

Contents :
1. [Creation of SYNED input file (.json)](#syned)
1. [**on-axis** spectrum of an ideal planar undulator](#spectrum_axis)
1. [spectrum of an ideal planar undulator **through a slit**](#spectrum_slit)
1. [**on-axis** tuning curve of an ideal planar undulator](#tc_axis)
1. [tuning curve of an ideal planar undulator **through a slit**](#tc_slit)
1. [Visualisations](#visu)

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '04/NOV/2024'
__changed__ = '04/NOV/2024'

import sys

import barc4sr.sr_undulator as und
import numpy as np
from barc4sr.aux_utils import (
                            ElectronBeam,
                            MagneticStructure,
                            SynchrotronSource,
)
from barc4xoc.plots import plot_tuning_curve
from scipy.constants import nano, pico

# checking we are using the correct python:
print(sys.executable)
print(sys.version)

%load_ext autoreload
%autoreload 2

In [ ]:
e0, ei, ef, dE = 0, 250, 5000, 0.5
ne = int((ef-ei)/dE)+1

aperture = 0.2e-3   # mrad
slit_position = 21  # m

window = np.tan(aperture/2)*slit_position*2

<a id="syned"></a>
### Creation of SYNED input file (.json)

[Back to the top](#toc)

In [ ]:
u52 = MagneticStructure(period_length=52.4e-3, number_of_periods=30)

In [ ]:
straight_section = ElectronBeam()
straight_section.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                            emittance=3.94*nano, coupling=1/100,
                            beta_x=3.9210, beta_y=2.1191, 
                            alpha_x=0.0514, alpha_y=0.1430,
                            eta_x=0.1632, eta_y=-0.0046,
                            etap_x=0.0015, etap_y=0.0018)
straight_section.print_rms()

u52.set_resonant_energy(350, 1, straight_section.energy_in_GeV, "v")

deimos = SynchrotronSource(straight_section, u52)
deimos.write_syned_config(r".\resources\oasys_soleil_hu52.json","Soleil - HU52")

In [ ]:
straight_section = ElectronBeam()
straight_section.from_twiss(energy=2.75, energy_spread=0.0906/100, current=0.500,
                  emittance_x=84.55*pico, emittance_y=25.36*pico,
                  beta_x=1.59, eta_x=0, etap_x=0, alpha_x=0,
                  beta_y=1.51, eta_y=0, etap_y=0, alpha_y=0)
straight_section.print_rms()

u52.set_resonant_energy(350, 1, straight_section.energy_in_GeV, "v")

deimos = SynchrotronSource(straight_section, u52)
deimos.write_syned_config(r".\resources\oasys_soleil-II_hu52.json","Soleil-II - HU52")

<a id="spectrum_axis"></a>
### Calculation of the **on-axis** spectrum of an ideal planar undulator
[Back to the top](#toc)

In [ ]:
soleil_u52_SOA = und.spectrum(file_name=".\\results\source\SOLEIL-U52mm_on_axis",
                        json_file=".\\resources\\oasys_soleil_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        hor_slit=1e-23,
                        ver_slit=1e-23)

In [ ]:
soleil_ii_u52_SOA = und.spectrum(file_name=".\\results\source\SOLEIL-II-U52mm_on_axis",
                        json_file=".\\resources\\oasys_soleil-ii_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        hor_slit=1e-23,
                        ver_slit=1e-23)

<a id="spectrum_slit"></a>
### Calculation of the on spectrum of an ideal planar undulator **through a slit**
[Back to the top](#toc)

In [ ]:
soleil_u52_STS = und.spectrum(file_name=".\\results\source\SOLEIL-U52mm_slit",
                        json_file=".\\resources\\oasys_soleil_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        hor_slit=window,
                        ver_slit=window,
                        parallel=True)

In [ ]:
soleil_ii_u52_STS = und.spectrum(file_name=".\\results\source\SOLEIL-II-U52mm_slit",
                        json_file=".\\resources\\oasys_soleil-ii_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        hor_slit=window,
                        ver_slit=window,
                        parallel=True)

<a id="tc_axis"></a>
### Calculation of the **on-axis** tuning curve of an ideal planar undulator
[Back to the top](#toc)

In [ ]:
e0, ei, ef, dE = 0, 250, 5000, 5
ne = int((ef-ei)/dE)+1

In [ ]:
soleil_u52_TCOA = und.tuning_curve(file_name=".\\results\source\SOLEIL-U52mm_on_axis",
                        json_file=".\\resources\\oasys_soleil_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        nHarmMax = 14,
                        Kmax = u52.K_vertical,
                        hor_slit=1e-23,
                        ver_slit=1e-23,
                        parallel=True)

In [ ]:
soleil_ii_u52_TCOA = und.tuning_curve(file_name=".\\results\source\SOLEIL-II-U52mm_on_axis",
                        json_file=".\\resources\\oasys_soleil-ii_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        nHarmMax = 14,
                        Kmax = u52.K_vertical,
                        hor_slit=1e-23,
                        ver_slit=1e-23,
                        parallel=True)

<a id="tc_slit"></a>
### Calculation of the tuning curve of an ideal planar undulator **through a slit**
[Back to the top](#toc)

In [ ]:
soleil_u52_TCTS = und.tuning_curve(file_name=".\\results\source\SOLEIL-U52mm_slit",
                        json_file=".\\resources\\oasys_soleil_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        nHarmMax = 14,
                        Kmax = u52.K_vertical,
                        hor_slit=window,
                        ver_slit=window,
                        parallel=True)

In [ ]:
soleil_ii_u52_TCTS = und.tuning_curve(file_name=".\\results\source\SOLEIL-II-U52mm_slit",
                        json_file=".\\resources\\oasys_soleil-ii_hu52.json",
                        photon_energy_min=ei,
                        photon_energy_max=ef,
                        photon_energy_points=ne,
                        observation_point=slit_position,
                        nHarmMax = 14,
                        Kmax = u52.K_vertical,
                        hor_slit=window,
                        ver_slit=window,
                        parallel=True)

<a id="visu"></a>
### Visualisations
[Back to the top](#toc)

In [ ]:
plot_tuning_curve(soleil_u52_TCOA, unit="[ph/s/mm²/0.1%bw]")

In [ ]:
plot_tuning_curve(soleil_u52_TCOA, unit="[ph/s/mm²/0.1%bw]")

In [ ]:
plot_tuning_curve(soleil_ii_u52_TCOA, unit="[ph/s/mm²/0.1%bw]")

In [ ]:
plot_tuning_curve(soleil_u52_TCTS, unit="[ph/s/0.1%bw]")

In [ ]:
plot_tuning_curve(soleil_ii_u52_TCTS, unit="[ph/s/0.1%bw]")